In [40]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import cvxopt
import csv
import pandas as pd
import numpy as np
import random
from random import randint
import time
from tqdm import tqdm, tqdm_notebook
import math
import pickle
from cvxopt import solvers
from cvxopt import matrix
from scipy.spatial.distance import pdist, cdist
from sklearn.svm import SVC
from tqdm import trange
from sklearn.model_selection import StratifiedKFold
from joblib import Parallel,delayed
import emot

In [94]:
train= pd.read_csv('fashion_mnist/train.csv',header=None).to_numpy()
val= pd.read_csv('fashion_mnist/val.csv',header=None).to_numpy()
test= pd.read_csv('fashion_mnist/test.csv',header=None).to_numpy()

In [95]:
def load_train_data(a,b):
    train_data = np.array([train[i,:] for i in range(train.shape[0]) if train[i,-1]== a or train[i,-1]==b ] )    
    X = train_data[:, :-1]/255
    m= X.shape[0]
    Y1 = train_data[:,-1].reshape((m,1))
    Y1[Y1==a] = -1
    Y1[Y1==b] = 1
    return X,Y1

In [96]:
def load_data():
    X = train[:, :-1]/255
    m= X.shape[0]
    Y1 = train[:,-1].reshape((m,1))

    X_val = val[:, :-1]/255
    m_val= X_val.shape[0]
    Y1_val = val[:,-1].reshape((m_val,1))

    X_test = test[:, :-1]/255
    m_test= X_test.shape[0]
    Y1_test = test[:,-1].reshape((m_test,1))

    return X_val, Y1_val, X_test, Y1_test, X,Y1

In [ ]:
def find_accuracy(y, y_pred):
    correct =0  
    for i in range(len(y)): 
        if y[i] == y_pred[i]:
            correct +=1
    return (correct/len(y)) *100

In [ ]:
def svm_gaussian(X,Y):
    m= X.shape[0]
    t =cdist(X,X,'euclidean')
    K = np.exp((t**2) * (-0.05))
    YY = Y@Y.T
    P=matrix(np.multiply(K,YY))
    q=matrix(np.ones([m,1])*-1)
    I = np.identity(m)
    I_neg = np.identity(m)*-1
    G = matrix(np.concatenate([I,I_neg]))
    A=matrix(Y.reshape((1,m)))
    b = matrix([0], tc='d')
    Y=Y.reshape(m)
    h = matrix(np.concatenate((np.ones([m,1]),np.zeros([m,1]))))  
    solvers.options['show_progress'] = False
    sol = solvers.qp(P,q,G,h,A,b)
    alpha=np.array(sol['x'])   
    SV_i =[i for i in range(len(alpha)) if alpha[i] >=1e-6]
    alpha1= np.array([alpha[i] for i in SV_i])
    SV_X = np.array([X[i] for i in SV_i])
    SV_Y = np.array([Y[i] for i in SV_i]).reshape((SV_X.shape[0],1))   
    X_pos= np.array([X[i] for i in range(len(Y)) if Y[i]==1.0])
    X_neg= np.array([X[i] for i in range(len(Y)) if Y[i]==-1.0])
    print(X_neg.shape)
    t_neg= cdist(SV_X,X_neg,'euclidean')
    K_neg= np.exp((t_neg**2) * (-0.05))
    t_pos= cdist(SV_X,X_pos,'euclidean')
    K_pos= np.exp((t_pos**2) * (-0.05))
    X_neg_k =np.dot(K_neg.T,(SV_Y*alpha1))
    X_pos_k =np.dot(K_pos.T,(SV_Y*alpha1))
    b = -(X_neg_k.max()+X_pos_k.min())/2
    print(alpha1.shape,SV_X.shape,X_pos.shape, X_neg.shape,  X_neg_k.shape)
    return [alpha1,b,SV_X, SV_Y]

In [ ]:
# X,Y=load_train_data(0,3)
# svm_gaussian(X,Y)

In [ ]:
classes = list(range(10))

In [ ]:
# l_model=[]
# for i in trange(10):
#     for j in trange(i+1, 10):
#         X,Y=load_train_data(classes[i],classes[j])
#         print(classes[i],classes[j])
#         model= svm_gaussian(X,Y)
#         l_model.append(model)

In [ ]:
# models_pickle = open("/content/drive/My Drive/Colab Notebooks/ML/ass2_part2/models_pickle",'wb')
# pickle.dump(l_model, models_pickle)


In [ ]:
file= open("/content/drive/My Drive/Colab Notebooks/ML/ass2_part2/models_pickle",'rb')
models = pickle.load(file)
file.close()
count =0
for i in range(10):
    for j in range(i+1, 10):
        models[count].append(i)
        models[count].append(j)
        count+=1
models = np.array(models)

In [ ]:
# class_pred=[]
# dict={}
# for i in range(10):
#     dict[i]= 0 
# for i in range(45):
#   class_pred.append(dict)
# class_pred[0][9]

In [ ]:
def find_max_class(dict):
    max =0
    max_key=-1
    for key in dict:
        if(dict[key][0]>max):
        max = dict[key][0]
        max_key=key
    elif (dict[key][0]==max):
        if(dict[key][1]>dict[max_key][1]):
        max_key = key
    return max_key

In [ ]:
def predict_gaussian(X):
    count=0;
    print(X.shape) 
    class_pred=[]
    for i in range(X.shape[0]):
    dict={}
    for i in range(10):
        dict[i]= [0,0] 
    class_pred.append(dict)
    for i in trange(45):
    [alpha,b,SV_X, SV_Y,p,q]=models[i]
    print(p,q)
    t= cdist(SV_X,X,'euclidean')
    K= np.exp((t**2) * (-0.05))
    pred = np.dot(K.T,(SV_Y*alpha))+b
    pred= pred.tolist()
    Y_pred =[]
    for j in range(len(pred)) :
        if pred[j][0] >0:         
            class_pred[j][q][0]+=1
              class_pred[j][q][1]+=pred[j][0]
        else :
            class_pred[j][p][0]+=1
            class_pred[j][p][1]+=abs(pred[j][0])
    for i in range(X.shape[0]):
        Y_pred.append(find_max_class(class_pred[i])) 
    print(class_pred[0])
    print(class_pred[2])
    return Y_pred

In [63]:
# def print_confusion_martix(Y_actual, Y_pred):
#     m = np.zeros((Y_actual.shape[0], Y_pred.shape[0]))
#     forY_actualalalrange(len(Y_actual)):
#         m[Y_actual][Y_pred]+=1
#     return m

In [100]:
X_val, Y_val, X_test, Y_test, X_train, Y_train=load_data()
print(X_train.shape)

(22500, 784)


In [ ]:
pred=predict_gaussian(X_val)

# accuracy on validation -  84.96
 

In [ ]:
find_accuracy(Y_val, pred)

In [ ]:
pred_test=predict_gaussian(X_test)

#accuracy on test data- 85.08

In [ ]:
find_accuracy(Y_test, pred_test)

In [ ]:
pred_train=predict_gaussian(X_train)

# Accuracy on train data = 96.52

In [ ]:
find_accuracy(Y_train, pred_train)

# 2(b) Using  Scikit learn

In [ ]:
skl_model=[]
for i in trange(10):
    for j in trange(i+1, 10):
        X,Y=load_train_data(classes[i],classes[j])
        print(classes[i],classes[j])
        svc = SVC(kernel='rbf', gamma=0.05, C=1)
        svc.fit(X, np.ravel(Y ,order = 'C'))
        skl_model.append(svc)

In [ ]:
# models_pickle_svc = open("/content/drive/My Drive/Colab Notebooks/ML/ass2_part2/models_pickle_svc",'wb')
# pickle.dump(skl_model, models_pickle_svc)

In [ ]:
# file1= open("/content/drive/My Drive/Colab Notebooks/ML/ass2_part2/models_pickle_svc",'rb')
# models_svc = pickle.load(file1)

In [ ]:
file= open("/content/drive/My Drive/Colab Notebooks/ML/ass2_part2/models_pickle_svc",'rb')
models = pickle.load(file)
print(type(models))
file.close()
classes=[]
count =0
for i in range(10):
    for j in range(i+1, 10):
        classes.append((i,j))
models = np.array(models)

In [ ]:
def predict_gaussian_svm(X):
    count=0;
    print(X.shape) 
    class_pred=[]
    Y_pred_class=[]
    for i in range(X.shape[0]):
    dict={}
    for i in range(10):
        dict[i]= [0,0] 
    class_pred.append(dict)
    for i in trange(45):
        (p,q)=classes[i]
        modeli=models[i]
        Y_pred = modeli.predict(X)
        Y_scores= modeli.decision_function(X)
        for j in range(len(Y_pred)) :
            if Y_pred[j] ==1:         
                class_pred[j][q][0]+=1
                class_pred[j][q][1]+=Y_scores[i]
            elif Y_pred[j] ==-1:
                class_pred[j][p][0]+=1
                class_pred[j][p][1]+=abs(Y_scores[i])
    for i in range(X.shape[0]):
        Y_pred_class.append(find_max_class(class_pred[i])) 
    print(class_pred[0])
    print(class_pred[2])
    return Y_pred_class

In [ ]:
pred=predict_gaussian_svm(X_val)
find_accuracy(Y_val, pred)

In [ ]:
find_accuracy(Y_val, pred)

In [ ]:
pred=predict_gaussian_svm(X_test)
find_accuracy(Y_test, pred)

In [ ]:
pred=predict_gaussian_svm(X_train)
find_accuracy(Y_train, pred)

## d) 5-fold cross validation 

In [82]:
X = train[:,:-1]/255 
Y = train[:, -1]
print(X.shape, Y.shape)
k=5
models=[]
C_values = [1e-5,1e-3,1, 5 ,10]
skf = StratifiedKFold(n_splits=k)
skf.get_n_splits(X, Y)

(22500, 784) (22500,)


In [84]:
def train_svc_multiclass(i):
    count=0
    avg_acc=0
    for train_index, test_index in skf.split(X, Y):
        count+=1
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]  
        svc = SVC(kernel='rbf', gamma=0.05, C=i, decision_function_shape='ovo')
        svc.fit(X_train, Y_train.ravel())
        print('done')
        models.append([svc,i,count])
        score= svc.score(X_test, Y_test)
        avg_acc = avg_acc+ score
        print(score, i,count)
    return avg_acc/k

In [85]:
s = time.time()
accuracy_on_c=Parallel(n_jobs=5)(delayed(train_svc_multiclass)(i) for i in C_values )
print(time.time()-s)

4120.884063959122


k fold accuracy [0.5664444444444444, 0.5664444444444444, 0.8787111111111111, 0.8844, 0.8842666666666666]

In [86]:
print(accuracy_on_c)
k_fold_acc = open('k_fold_acc','wb')
pickle.dump(accuracy_on_c, k_fold_acc)

[0.5664444444444444, 0.5664444444444444, 0.8787111111111111, 0.8844, 0.8842666666666666]


In [103]:
test_accuracy=[]
C_values = [1e-5,1e-3,1, 5 ,10]
for i in C_values:
    svc = SVC(kernel='rbf', gamma=0.05, C=i, decision_function_shape='ovo')
    svc.fit(X, Y.ravel())
    test_accuracy.append(svc.score(X_test, Y_test))

test accuracy [0.5736, 0.5736, 0.8808, 0.8828, 0.8824]

In [104]:
test_accuracy

[0.5736, 0.5736, 0.8808, 0.8828, 0.8824]